# Train BonDNet 

In this notebook, we show how to train the BonDNet graph neural network model for bond dissociation energy (BDE) prediction. We only show how to train on CPUs. See [train_bde_distributed.py](./) for a script for training on GPUs (a single GPU or distributed training on multiple GPUs). 

In [1]:
import torch
import time, wandb
import numpy as np 
from tqdm import tqdm
from sklearn.metrics import r2_score
from torchmetrics import R2Score
from torch.optim.lr_scheduler import ReduceLROnPlateau

from bondnet.model.metric import EarlyStopping
from bondnet.data.dataset import ReactionNetworkDatasetGraphs
from bondnet.data.dataloader import DataLoaderReactionNetwork
from bondnet.data.dataset import train_validation_test_split
#from bondnet.scripts.create_label_file import read_input_files
#from bondnet.model.metric import WeightedL1Loss, WeightedMSELoss
from bondnet.utils import seed_torch, pickle_dump, parse_settings
from bondnet.model.training_utils import (
    evaluate, 
    evaluate_classifier, 
    train, 
    train_classifier, 
    load_model, 
    evaluate_r2, 
    get_grapher
)
seed_torch()

/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)

1.12.1


## Dataset 

We work with a small dataset consisting of 200 BDEs for netural and charged molecules. The dataset is specified in three files:
- `molecules.sdf` This file contains all the molecules (both reactants and products) in the bond dissociation reactions. The molecules are specified in SDF format. 
- `molecule_attributes.yaml` This file contains extra molecular attributes (charges here) for molecules given in `molecules.sdf`. Some molecular attributes can be inferred from its SDF block, and they are overrode by the attributes specified in the `molecule_attributes.yaml` file.  
- `reactions.csv` This file list the bond dissociation reations formed by the molecules given in `molecules.sdf`. Each line lists the reactant, products, and BDE of a reaction. The reactant and products are specified by their index in `molecules.sdf`. 

See [here](./examples/train) for the three files used in this notebook. 

#### Grapher 

BondNet is graph neutral network model that takes atom features (e.g. atom type), bond features (e.g. whether a bond is in a ring), and global features (e.g. total charge) as input. We extract the features for a molecule using a grapher.

#### Read dataset 

Let's now read the dataset and featurize the molecules using the above defined grapher. The dataset is split into a training set (80%), validation set (10%), and test set (10%). We will train our model using the training set, stop the training using the validation set, and report error on the test set. 

In [3]:
settings_file = './settings.txt'
best = 1e10
feature_names = ["atom", "bond", "global"]
dict_train = parse_settings(settings_file)
path_mg_data = "../../../dataset/mg_dataset/20220613_reaction_data.json"
    

using the following settings:
----------------------------------------
Small Dataset?: False
restore: True
distributed: False
on gpu: False
num gpu: 1
hyperparam save file: ./hyper.pkl
dataset state dict: home/santiagovargas/Documents/Dataset/mg/dataset_state_dict.pkl
model dir /home/santiagovargas/Documents/Dataset/mg/
classifier False
batch size: 256
epochs: 100
lr: 0.000100
weight decay: 0.000
early_stop: True
scheduler: True
transfer_epochs: 100
transfer: False
loss: False
categories: 3
embedding size: 24
fc layers: 2
fc hidden layer: [128, 64]
gated layers: 3
gated hidden layers: [64, 64, 64]
num lstm iters: 6
num lstm layer: 3
gated fc layers: 2
fc activation: ReLU
fc batch norm: 0
fc dropout: 0.00
gated activation: ReLU
gated dropout: 0.10
gated batch norm: True
gated graph norm: 0
gated resid: True
----------------------------------------


In [4]:
device = None
if(device == None):
    if dict_train["on_gpu"]:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        dict_train["gpu"] = device
    else:
        device = torch.device("cpu")
        dict_train["gpu"] = "cpu"
else: 
    dict_train["gpu"] = device
    device = torch.device("cpu")
path_mg_data = '../dataset/mg_dataset/20220613_reaction_data.json'
dataset = ReactionNetworkDatasetGraphs(
grapher=get_grapher(), 
file=path_mg_data, 
out_file="./", 
target = 'ts', 
classifier = dict_train["classifier"], 
classif_categories=5, 
debug = True,
device =  device
)

reading file from: ../dataset/mg_dataset/20220613_reaction_data.json
rxn raw len: 500
Program finished in 13.24595564700212 seconds
.............failures.............
reactions len: 54
valid ind len: 54
bond break fail count: 		0
default fail count: 		446
sdf map fail count: 		0
product bond fail count: 	0
about to group and organize
number of grouped reactions: 54


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders



features: 132
labels: 54
molecules: 132
constructing graphs & features....
number of graphs valid: 132
number of graphs: 132


In [5]:
dataset

Dataset ReactionNetworkDatasetGraphs
Length: 54
Feature: atom, size: 12
Feature: bond, size: 13
Feature: global, size: 7
Feature: atom, name: ['total degree', 'is in ring', 'total H', 'chemical symbol', 'chemical symbol', 'chemical symbol', 'chemical symbol', 'ring size', 'ring size', 'ring size', 'ring size', 'ring size']
Feature: bond, name: ['metal bond', 'ring inclusion', 'ring size', 'ring size', 'ring size', 'ring size', 'ring size', 'conjugated', 'single', 'double', 'triple', 'aromatic', 'bond_len']
Feature: global, name: ['num atoms', 'num bonds', 'molecule weight', 'charge one hot', 'charge one hot', 'charge one hot', 'charge one hot']

In [6]:
trainset, valset, testset = train_validation_test_split(dataset, validation=0.2, test=0.2)
dataset_loader = DataLoaderReactionNetwork(dataset, batch_size=100,shuffle=True)
train_loader = DataLoaderReactionNetwork(trainset, batch_size=100,shuffle=True)
val_loader = DataLoaderReactionNetwork(valset, batch_size=len(valset), shuffle=False)
test_loader = DataLoaderReactionNetwork(testset, batch_size=len(testset), shuffle=False)
#test_ind = 3
#elements = [i['name'] for i in dataset.pandas_df.iloc[test_ind]['reactant_molecule_graph']['molecule']['sites']]
#print(elements)
#print(len(dataset.molecules))

## Model 

We create the BonDNet model by instantiating the `GatedGCNReactionNetwork` class and providing the parameters defining the model structure. 
- `embedding_size` The size to unify the atom, bond, and global feature length.
- `gated_num_layers` Number of graph to graph module to learn molecular representation. 
- `gated_hidden_size` Hidden layer size in the graph to graph modules. 
- `gated_activation` Activation function appleid after the hidden layers in the graph to graph modules. 
- `fc_num_layers` Number of hidden layers of the fully connected network to map reaction feature to the BDE. The reaction feature is obtained as the differece of the features between the products and the reactant. 
- `fc_hidden_size` Size of the hidden layers. 
- `fc_activation` Activation function applied after the hidden layers. 

There are other arguments (e.g. residual connection, dropout ratio, batch norm) that can be specified to fine control the model. See the documentation of the `GatedGCNReactionNetwork` for more information.  

In [7]:
from bondnet.model.gated_reaction_network import GatedGCNReactionNetwork

from bondnet.model.metric import EarlyStopping
from bondnet.data.dataset import ReactionNetworkDatasetGraphs
from bondnet.data.dataloader import DataLoaderReactionNetwork
from bondnet.data.dataset import train_validation_test_split
#from bondnet.scripts.create_label_file import read_input_files
#from bondnet.model.metric import WeightedL1Loss, WeightedMSELoss
from bondnet.utils import seed_torch, pickle_dump, parse_settings
from bondnet.model.training_utils import (
    evaluate, 
    evaluate_classifier, 
    train, 
    train_classifier, 
    load_model, 
    evaluate_r2, 
    get_grapher
)


model = GatedGCNReactionNetwork(
    in_feats=dataset.feature_size,
    embedding_size=24,
    gated_num_layers=2,
    gated_hidden_size=[64, 64, 64],
    gated_activation="ReLU",
    fc_num_layers=2,
    fc_hidden_size=[128, 64],
    fc_activation='ReLU'
)
#print(dict(model.named_parameters()))

## Train the model 

Before going to the main training loop, we define two functions: `train` and `evaluate` that will be used later. 

The `train` function optimizes the model parameters for an epoch. We note that our target BDEs are centered and then normalized by the standard deviation (done in the `ReactionNetworkDataset`.) So to measure the mean absolute error, we need to multiply the standard deviation back. This is acheived achieved by the `WeightedL1Loss` function passed as `metric_fn`.   

Now, we have all the ingredients to train the model. 

We optimize the model parameters by minimizing a mean squared error loss function using the `Adam` optimizer with a learning rate of `0.001`. Here we train the model for 20 epochs; save the best performing model that gets the smallest mean absolute error on the validation set; and finally test model performance on the test set. 

In [8]:
dict_train['in_feats'] = dataset.feature_size
model, optimizer, optimizer_transfer = load_model(dict_train)

for epoch in tqdm(range(dict_train['transfer_epochs'])):
    loss_transfer, train_acc_transfer = train(
        model, 
        feature_names, 
        train_loader, 
        optimizer_transfer, 
        device = dict_train["gpu"]
    )
    val_acc_transfer = evaluate(
        model, 
        feature_names, 
        val_loader, 
        device = dict_train["gpu"]
    )


  0%|          | 0/100 [00:00<?, ?it/s]

atom
7
torch.Size([7, 64])
bond
6
torch.Size([6, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.Size([8, 64])
global
atom
11
torch.Size([11, 64])
bond
11
torch.Size([11, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.Size([8, 64])
global
atom
11
torch.Size([11, 64])
bond
11
torch.Size([11, 64])
global
atom
13
torch.Size([13, 64])
bond
13
torch.Size([13, 64])
global
atom
7
torch.Size([7, 64])
bond
7
torch.Size([7, 64])
global
atom
11
torch.Size([11, 64])
bond
11
torch.Size([11, 64])
global
atom
8
torch.Size([8, 64])
bond
9
torch.Size([9, 64])
global
atom
13
torch.Size([13, 64])
bond
13
torch.Size([13, 64])
global
atom
10
torch.Size([10, 64])
bond
10
torch.Size([10, 64])
global
atom
8
torch.Size([8, 64])
bond
7
torch.Size([7, 64])
global
atom
8
torch.Size([8, 64])
bond
9
torch.Size([9, 64])
global
atom
14
torch.Size([14, 64])
bond
14
torch.Size([14, 64])
global
atom
18
torch.Size([18, 64])
bond
21
torch.Size([21, 64])
global
atom
19
torch.Size([19, 64])
bond
21
torch.Size([21, 

  1%|          | 1/100 [00:01<02:16,  1.38s/it]

atom
15
torch.Size([15, 64])
bond
16
torch.Size([16, 64])
global
atom
10
torch.Size([10, 64])
bond
10
torch.Size([10, 64])
global
atom
11
torch.Size([11, 64])
bond
11
torch.Size([11, 64])
global
atom
23
torch.Size([23, 64])
bond
24
torch.Size([24, 64])
global
atom
12
torch.Size([12, 64])
bond
13
torch.Size([13, 64])
global
atom
19
torch.Size([19, 64])
bond
19
torch.Size([19, 64])
global
atom
16
torch.Size([16, 64])
bond
16
torch.Size([16, 64])
global
atom
7
torch.Size([7, 64])
bond
6
torch.Size([6, 64])
global
atom
15
torch.Size([15, 64])
bond
14
torch.Size([14, 64])
global
atom
19
torch.Size([19, 64])
bond
19
torch.Size([19, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.Size([8, 64])
global
atom
19
torch.Size([19, 64])
bond
21
torch.Size([21, 64])
global
atom
13
torch.Size([13, 64])
bond
13
torch.Size([13, 64])
global
atom
11
torch.Size([11, 64])
bond
12
torch.Size([12, 64])
global
atom
17
torch.Size([17, 64])
bond
17
torch.Size([17, 64])
global
atom
11
torch.Size([11, 64])
bond

  2%|▏         | 2/100 [00:01<01:13,  1.34it/s]

...

 71%|███████   | 71/100 [00:22<00:09,  3.16it/s]

atom
16
torch.Size([16, 64])
bond
15
torch.Size([15, 64])
global
atom
11
torch.Size([11, 64])
bond
11
torch.Size([11, 64])
global
atom
8
torch.Size([8, 64])
bond
7
torch.Size([7, 64])
global
atom
11
torch.Size([11, 64])
bond
11
torch.Size([11, 64])
global
atom
13
torch.Size([13, 64])
bond
13
torch.Size([13, 64])
global
atom
19
torch.Size([19, 64])
bond
21
torch.Size([21, 64])
global
atom
14
torch.Size([14, 64])
bond
14
torch.Size([14, 64])
global
atom
17
torch.Size([17, 64])
bond
16
torch.Size([16, 64])
global
atom
15
torch.Size([15, 64])
bond
14
torch.Size([14, 64])
global
atom
7
torch.Size([7, 64])
bond
6
torch.Size([6, 64])
global
atom
8
torch.Size([8, 64])
bond
9
torch.Size([9, 64])
global
atom
11
torch.Size([11, 64])
bond
12
torch.Size([12, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.Size([8, 64])
global
atom
13
torch.Size([13, 64])
bond
13
torch.Size([13, 64])
global
atom
10
torch.Size([10, 64])
bond
10
torch.Size([10, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.S

 71%|███████   | 71/100 [00:23<00:09,  3.06it/s]

atom
13
torch.Size([13, 64])
bond
13
torch.Size([13, 64])
global
atom
22
torch.Size([22, 64])
bond
21
torch.Size([21, 64])
global
atom
19
torch.Size([19, 64])
bond
21
torch.Size([21, 64])
global
atom
15
torch.Size([15, 64])
bond
14
torch.Size([14, 64])
global
atom
8
torch.Size([8, 64])
bond
7
torch.Size([7, 64])
global
atom
8
torch.Size([8, 64])
bond
9
torch.Size([9, 64])
global
atom
11
torch.Size([11, 64])
bond
11
torch.Size([11, 64])
global
atom
7
torch.Size([7, 64])
bond
6
torch.Size([6, 64])
global
atom
7
torch.Size([7, 64])
bond
7
torch.Size([7, 64])
global
atom
16
torch.Size([16, 64])
bond
17
torch.Size([17, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.Size([8, 64])
global
atom
9
torch.Size([9, 64])
bond
8
torch.Size([8, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.Size([8, 64])
global
atom
14
torch.Size([14, 64])
bond
14
torch.Size([14, 64])
global
atom
10
torch.Size([10, 64])
bond
10
torch.Size([10, 64])
global
atom
8
torch.Size([8, 64])
bond
8
torch.Size([8, 64])

KeyboardInterrupt: 